<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>Cuts Optimization using Extra Gradient Boosting
<br></p><br>

Over the last years, **Machine Learning** tools have been successfully applied to problems in high-energy physics. For example, for the classification of physics objects. Supervised machine learning algorithms allow for significant improvements in classification problems by taking into account observable correlations and by learning the optimal selection from examples, e.g. from Monte Carlo simulations.


# Importing the Libraries

**Numpy** is a powerful library that makes working with python more efficient, so we will import it and use it as np in the code. **Pandas** is another useful library that is built on numpy and has two great objects *series* and *dataframework*. Pandas works great for *data ingestion* and also has *data visualization* features. From **Hipe4ml** we import **TreeHandler** and with the help of this function we will import our *Analysis Tree* to our notebook.

**Matplotlib** comes handy in plotting data while the machine learning is performed by **XGBOOST**. We will import data splitter from **Scikit-learn** as *train_test_split*. **Evaluation metrics** such as *confusion matrix*, *Receiver operating characteristic (ROC)*, and *Area Under the Receiver Operating Characteristic Curve (ROC AUC)*  will be used to asses our models.

A **Confusion Matrix** $C$ is such that $C_{ij}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$. Thus in binary classification, the count of true positives is $C_{00}$, false negatives $C_{01}$,false positives is $C_{10}$, and true neagtives is $C_{11}$.

If $ y^{'}_{i} $ is the predicted value of the $ i$-th sample and $y_{i}$ is the corresponding true value, then the fraction of correct predictions over $ n_{samples}$ is defined as 
$$
True \: positives (y,y^{'}) =  \sum_{i=1}^{n_{samples} } 1 (y^{'}_{i} = y_{i}=1)
$$ 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hipe4ml.model_handler import ModelHandler
from hipe4ml.tree_handler import TreeHandler
from hipe4ml import plot_utils

import xgboost as xgb
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import plot_tree
from sklearn.tree import export_graphviz

from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform

from numpy import sqrt, log
from numpy import argmax
import weakref 

from bayes_opt import BayesianOptimization

import uproot
from root_pandas import read_root

In [ ]:
#To save some memory we will delete unused variables
class TestClass(object): 
    def check(self): 
        print ("object is alive!") 
    def __del__(self): 
        print ("object deleted") 

In [ ]:
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(8)

In [ ]:
import sys
import resource
import uproot

def peak_memory_usage():
    """Return peak memory usage in MB"""
    mem = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    factor_mb = 1 / 1024
    if sys.platform == "darwin":
        factor_mb = 1 / (1024 * 1024)
    return mem * factor_mb

input_tree = uproot.open('/home/shahid/cbmsoft/Data/10k_events_PFSimplePlainTree.root:PlainTree')
branches = input_tree.arrays()
print(peak_memory_usage())

# Importing the data
CBM has a modified version of the cern's root software and it contains the simulated setup of CBM. Normally, a model generated input file, for example a URQMD 12 AGeV, is passed through different macros. These macros represent the CBM setup and it is like taking particles and passing them through a detector. These particles are registered as hits in the setup. Then particles' tracks are reconstructed from these hits using cellular automaton and Kalman Filter mathematics.


CBM uses the **tree** format of cern root to store information. To reduce the size of these root files a modified tree file was created by the name of Analysis tree. This Analysis tree file contains most of the information that we need for physics analysis. 

In this example, we download three Analysis Trees. The first one contains mostly background candidates for lambda i.e. protons and pions which do not come from a lambda. The second file contains mostly signal candidates of lamba i.e. it contains protons and pions which come from a lambda decay. The third one contains 10k events generated using URQMD generator with 12 AGeV energy.

In [ ]:
# We import the tree 'PlainTree' from root file into our jupyter notebook
signal = uproot.open('/home/shahid/cbmsoft/Data/PFSimplePlainTreeSignal.root:PlainTree', num_workers=8).arrays(library='pd')
# We only select lambda candidates
sgnal = signal[(signal['LambdaCandidates_is_signal']==1) & (signal['LambdaCandidates_mass']>1.108) & (signal['LambdaCandidates_mass']<1.1227)

del signal

In [ ]:
df_original1 = read_root('/home/shahid/cbmsoft/Data/cbm/apr20_fr_18.2.1_fs_jun19p1/urqmd_pluto/auau/12agev/mbias/sis100_electron/TGeant4/analysis_tree/PFSimpleOutput.1.root','sTree')


In [ ]:
signal = read_root('/home/shahid/cbmsoft/Data/PFSimplePlainTreeSignal.root','PlainTree')
sgnal = signal[(signal['LambdaCandidates_is_signal']==1) & (signal['LambdaCandidates_mass']>1.108)
               & (signal['LambdaCandidates_mass']<1.1227)]


background = read_root('/home/shahid/cbmsoft/Data/PFSimplePlainTreeBackground.root','PlainTree')
bg = background[(background['LambdaCandidates_is_signal'] == 0)
                & ((background['LambdaCandidates_mass'] > 1.07)
                & (background['LambdaCandidates_mass'] < 1.108) | (background['LambdaCandidates_mass']>1.1227) 
                   & (background['LambdaCandidates_mass'] < 2.00))]


del signal
del background

In [ ]:
df_original = read_root('/home/shahid/cbmsoft/Data/10k_events_PFSimplePlainTree.root','PlainTree')
#df_original = df_original1[(df_original1['LambdaCandidates_is_signal']==1) & (df_original1['LambdaCandidates_mass']>1.108) & (df_original1['LambdaCandidates_mass']<1.1227)]

In [ ]:
input_tree = uproot.open('/home/shahid/cbmsoft/Data/10k_events_PFSimplePlainTree.root:PlainTree', decompression_executor=executor, interpretation_executor=executor)

In [ ]:
branches = input_tree.arrays(library='pd', decompression_executor=executor, interpretation_executor=executor)

In [ ]:
del input_tree

In [ ]:
input_tree_signal = TreeHandler('/home/shahid/cbmsoft/Data/PFSimplePlainTreeSignal.root','PlainTree')
sgnal = input_tree_signal.get_data_frame()

In [ ]:
# We import three root files into our jupyter notebook
input_tree_signal = TreeHandler('/home/shahid/cbmsoft/Data/PFSimplePlainTreeSignal.root','PlainTree')
# We only select lambda candidates
true_signal = input_tree_signal.get_subset('LambdaCandidates_is_signal == 1 & LambdaCandidates_mass>1.108 & LambdaCandidates_mass<1.1227')
# Then we store the information in a dataframe object, sgnal, of Pandas library
sgnal = true_signal.get_data_frame()

# Similarly for the background
input_tree_background = TreeHandler('/home/shahid/cbmsoft/Data/PFSimplePlainTreeBackground.root','PlainTree')
true_background = input_tree_background.get_subset('LambdaCandidates_is_signal == 0 & 1.07< LambdaCandidates_mass < 1.108 or 1.1227 < LambdaCandidates_mass < 2.00')
bg=true_background.get_data_frame()

# The 10k events data set
input_tree = TreeHandler('/home/shahid/cbmsoft/Data/10k_events_PFSimplePlainTree.root','PlainTree')
df_original = input_tree.get_data_frame()

In [ ]:
del input_tree_signal
del true_signal
del input_tree_background
del true_background
del input_tree

In [ ]:
#The labels of the columns in the df data frame are having the prefix LambdaCandidates_ so we rename them
new_labels= ['chi2geo', 'chi2primneg', 'chi2primpos', 'chi2topo', 'cosineneg',
       'cosinepos', 'cosinetopo', 'distance', 'eta', 'l', 'ldl',
       'mass', 'p', 'pT', 'phi', 'px', 'py', 'pz', 'rapidity',
             'x', 'y', 'z', 'daughter1id', 'daughter2id', 'isfrompv', 'pid', 'issignal']

df_original.columns=new_labels
sgnal.columns = new_labels
bg.columns = new_labels

#Let's see how the dataframe object df looks like
#df_original

The above data frame object has some columns/features and for them at the very last column the true Monte Carlos information is available. This MC information tells us whether this reconstructed particle was originally produced as a decaying particle or not. So a value of 1 means that it is a true candidate and 0 means that it is not.

# Data Cleaning
Sometimes a data set contains entries which do not make sense. For example, infinite values or NaN entries. We clean the data by removing these entries. Ofcourse, we lose some data points but these outliers sometimes cause problems when we perform analysis. 

Since our experiment is a fixed target experiment so there are certain constraints which have to be applied on the data as well.

In [ ]:
#Creating a new data frame and saving the results in it after cleaning of the original dfs
#Also keeping the original one
bcknd = clean_df(bg)
signal = clean_df(sgnal)
df_clean = clean_df(df_original)

del bg
del sgnal

In [ ]:
del df_original

In [ ]:
df_clean_signal_mask = df_clean['issignal']>0
df_clean_signal = df_clean[df_clean_signal_mask]
df_clean_bac_mask = df_clean['issignal']==0
df_clean_bac = df_clean[df_clean_bac_mask]

In [ ]:
leg_labels = ['background', 'signal']
vars_to_draw = df_clean.columns

ax = plot_utils.plot_distr([df_clean_bac, df_clean_signal], vars_to_draw, bins=100, labels=leg_labels, log=True, density=True, figsize=(30, 20), alpha=0.3, grid=False)
plt.tight_layout()
plt.savefig("hits.png")

In [ ]:
df_clean_signal.columns[27]

In [ ]:
l=27
fig, axs = plt.subplots(figsize=(20, 10))
df_clean_signal.iloc[:,l].plot.hist(bins = 1000,   alpha=0.2,facecolor='red', grid=False)
df_clean_bac.iloc[:,l].plot.hist(bins = 1000,  alpha=0.2,facecolor='blue',grid=False)
plt.yscale("log")
plt.title(df_clean_signal.columns[l] + ' MC', fontsize = 15)
plt.xlabel(df_clean_signal.columns[l])
plt.legend(('signal','background'), fontsize = 15)
plt.tight_layout()
fig.savefig("hists.png")

In [ ]:
df_clean['xgb_preds'].describe()

In [ ]:
df_clean['distance'].describe()

In [ ]:
aa = df_clean['distance']+0.001
aa.plot.hist(bins=300, range=(0,0.1))
plt.yscale('log')

In [ ]:
a = np.random.normal(loc=0, scale=0.115810, size=(120093,1))
new_series = pd.DataFrame(a)
new_series.columns=['gauss']
new_series1 = (0.11+asas)

In [ ]:
mask12= (signal['distance']>=1)  & (signal['distance']<3)
asas1=signal['distance'][mask12]
asas1.describe()
asas1.plot.hist(bins=300)
#plt.yscale('log')

In [ ]:
uniform_random_array = np.random.uniform(low=0.000011, high=11.969676, size=(120870,1))
uniform_random_df = pd.DataFrame(uniform_random_array)
uniform_random_df.plot.hist(bins=300)

# Selecting Background and Signal
Our sample contains a lot of background (2178718) and somewhat signal candidates (36203). For analysis we will use a signal set of 4000 candidates and a background set of 12000 candidates. The background and signal candidates will be selected by using MC information.

In [ ]:
# We randomly choose our signal set of 4000 candidates
signal_selected= signal.sample(n=90000)

#background = 3 times the signal is also done randomly
background_selected = bcknd

del signal
del bcknd

In [ ]:
#Let's combine signal and background
dfs = [signal_selected, background_selected]
df_scaled = pd.concat(dfs)

#del signal_selected
#del background_selected

In [ ]:
# Let's shuffle the rows randomly
df_scaled = df_scaled.sample(frac=1)
del dfs
# Let's take a look at the top 10 entries of the df
df_scaled.iloc[0:10,:]

In [ ]:
range1 = (1.075, 1.18)
fig, axs = plt.subplots(figsize=(10, 6))
#df_scaled['mass'].plot.hist(bins = 300, range=range1,grid=True,sharey=True)
background_selected['mass'].plot.hist(bins = 300, facecolor='yellow',grid=True,range=range1)
signal_selected['mass'].plot.hist(bins = 300, facecolor='magenta',grid=True, range=range1)
plt.ylabel("Counts", fontsize=15)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize= 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Test and Train Lambda Invariant Mass', fontsize = 15)
plt.legend(('Background', 'Signal'), fontsize = 15)
fig.tight_layout()
fig.savefig("signal_bac_invmass_MC.png")

# Creating Train and Test sets
To make machine learning algorithms more efficient on unseen data we divide our data into two sets. One set is for training the algorithm and the other is for testing the algorithm. If we don't do this then the algorithm can overfit and we will not capture the general trends in the data. 

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
cuts = [ 'chi2primneg', 'chi2primpos', 'chi2topo', 'cosinetopo', 'distance', 'ldl', 
         'x','y',
       'daughter2id']


x = df_scaled[cuts].copy()

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
cuts = [ 'chi2primneg', 'chi2primpos', 'ldl', 'distance', 'chi2geo','cosinepos']


x = df_scaled[cuts].copy()

In [ ]:
# The MC information is saved in this y variable
y =pd.DataFrame(df_scaled['issignal'], dtype='int')
type(y)

## Whole set

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
x_whole = df_clean[cuts].copy()
# The MC information is saved in this y variable
y_whole = pd.DataFrame(df_clean['issignal'], dtype='int')

# KFPF

In [ ]:
new_check_set= df_clean.copy()
new_check_set['new_signal']=0
mask1 = (new_check_set['chi2primpos'] > 18.4) & (new_check_set['chi2primneg'] > 18.4)

mask2 = (new_check_set['ldl'] > 5) & (new_check_set['distance'] < 1)

mask3 = (new_check_set['chi2geo'] < 3) & (new_check_set['cosinepos'] > 0) & (new_check_set['cosineneg'] > 0)

new_check_set = new_check_set[(mask1) & (mask2) & (mask3)] 

#After all these cuts, what is left is considered as signal, so we replace all the values in the 'new_signal'
# column by 1
new_check_set['new_signal'] = 1
signal_in_signal_mask = new_check_set['issignal']==1
signal_in_signal = new_check_set[signal_in_signal_mask]
signal_in_signal

<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>XGB Boost 
<br></p><br>

## Random search

In [ ]:

# Instantiate an XGBRegressor with default hyperparameter settings
xgbr = xgb.XGBRegressor(max_depth = 8, alpha=10,gamma =1)

# and compute a baseline to beat with hyperparameter optimization 
baseline = cross_val_score(xgbr, x, y, scoring='roc_auc').mean()

In [ ]:
baseline

In [ ]:
param_dist = { "n_estimators": range(113,120)}

rs = RandomizedSearchCV(xgbr, param_distributions=param_dist, 
                        scoring='roc_auc', n_iter=7)


a= rs.fit(x, y);

In [ ]:
rs.best_params_

## Grid search

In [ ]:
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

#from sklearn.preprocessing import Imputer

In [ ]:
#XGBoost hyper-parameter tuning
def hyperParameterTuning(x, y):
    param_tuning = {
        'learning_rate': [0.01, 0.02, 0.03,0.1],
        'max_depth': [3, 5, 8, 10,12,14],
        'n_estimators' : [100, 200,300,400, 500],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           scoring = 'roc_auc', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(x,y)

    return gsearch.best_params_

In [ ]:
hyperParameterTuning(x_train, y_train)

In [ ]:
xgb_model = XGBRegressor(
        objective = 'reg:squarederror',
        learning_rate = 0.1,
        max_depth = 8,
        n_estimators = 200)

%time xgb_model.fit(x_train, y_train, early_stopping_rounds=5, eval_set=[(x_test, y_test)], verbose=False)

y_pred_xgb = xgb_model.predict(x_test)

mae_xgb = roc_auc_score(y_test, y_pred_xgb)

print("MAE: ", mae_xgb)

## Bayesian
In order to find the best parameters of XGB for our data we use Bayesian optimization. Grid search and and random search could also do the same job but bayesian is more time efficient.

In [ ]:
from numba import vectorize

In [ ]:
from numba import jit
sum_array_numba = jit()(sum_array)
jitted = %timeit -o sum_array_numba(arr)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=324)

In [ ]:
dtrain = xgb.DMatrix(x_train, label = y_train)
dtest = xgb.DMatrix(x_whole, label = y_whole)
dtest1=xgb.DMatrix(x_test, label = y_test)

### Hyper parameters

*subsample* [default=1]
Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.
range: (0,1]

*eta* [default=0.3, alias: learning_rate]
Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative.
range: [0,1]


*gamma* [default=0, alias: min_split_loss]
Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.
range: [0,∞]


*alpha* [default=0, alias: reg_alpha]
L1 regularization term on weights. Increasing this value will make model more conservative.

*Lasso Regression* (Least Absolute Shrinkage and Selection Operator) adds “absolute value of magnitude” of coefficient as penalty term to the loss function.

In [ ]:
#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma, alpha, n_estimators ,learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'alpha':alpha,
              'n_estimators': n_estimators,
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'auc', 'nthread' : 7}
    cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    return  cv_result['test-auc-mean'].iloc[-1]



In [ ]:
#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (4, 10),
                                             'gamma': (0, 1),
                                            'alpha': (2,20),
                                             'learning_rate':(0,1),
                                             'n_estimators':(100,500)
                                            })

In [ ]:
#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=15, init_points=8, acq='ei')
#0.9952

In [ ]:
# best target so far 0.994872
print(xgb_bo.max)

# XGB models

## New models for treelite

In [ ]:
max_param = xgb_bo.max['params']
param= {'alpha': max_param['alpha'], 'gamma': max_param['gamma'], 'learning_rate': max_param['learning_rate'], 'max_depth': int(round(max_param['max_depth'],0)), 'n_estimators': int(round(max_param['n_estimators'],0)), 'objective': 'binary:logistic'}

In [ ]:
bst = xgb.train(param, dtrain)

In [ ]:
bst1= bst.predict(dtrain)

In [ ]:
bst_test = bst.predict(dtest1)

In [ ]:
def significance(y_true, y_predict, y_true1, y_predict1):
    roc_auc=roc_auc_score(y_true, y_predict)
    fpr, tpr, thresholds = roc_curve(y_true, y_predict,drop_intermediate=False ,pos_label=1)
    S0 = sqrt(2 * ((tpr + fpr) * log((1 + tpr/fpr)) - tpr))
    S0 = S0[~np.isnan(S0)]
    xi = argmax(S0)
    S0_best_threshold = (thresholds[xi])
    
    roc_auc1=roc_auc_score(y_true1, y_predict1)
    fpr1, tpr1, thresholds1 = roc_curve(y_true1, y_predict1,drop_intermediate=False ,pos_label=1)
    S01 = sqrt(2 * ((tpr1 + fpr1) * log((1 + tpr1/fpr1)) - tpr1))
    S01 = S01[~np.isnan(S01)]
    xi1 = argmax(S01)
    S0_best_threshold1 = (thresholds[xi1])
    
    fig, axs = plt.subplots(figsize=(15, 10), dpi = 100)
    plt.plot(fpr, tpr, linestyle=':',color='darkorange',label='ROC curve train (area = %0.4f)' % roc_auc)
    plt.plot(fpr1, tpr1, color='green',label='ROC curve test (area = %0.4f)' % roc_auc1)
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.scatter(fpr[xi], tpr[xi], marker='o', color='black', label= 'Best Threshold train set = '+"%.4f" % S0_best_threshold +'\n S0 = '+ "%.2f" % S0[xi])
    plt.scatter(fpr1[xi1], tpr1[xi1], marker='o', color='blue', label= 'Best Threshold test set = '+"%.4f" % S0_best_threshold1 +'\n S0 = '+ "%.2f" % S01[xi1])
    plt.xlabel('False Positive Rate', fontsize = 15)
    plt.ylabel('True Positive Rate', fontsize = 15)
    plt.legend(loc="lower right", fontsize = 15)
    plt.title('Receiver operating characteristic', fontsize = 15)
    plt.xlim([-0.01, 1.0])
    plt.ylim([0, 1.02])
    #axs.axis([-0.01, 1, 0.9, 1])
    fig.tight_layout()
    plt.show()

In [ ]:
significance(y_train, bst1,y_test, bst_test)

In [ ]:
df_clean['xgb_preds'] = bst.predict(dtest)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
df_clean['xgb_preds'].plot.hist(ax=ax, bins=300,facecolor='red',alpha = 0.3, bottom=0.001)
TP = df_clean[(df_clean['issignal']==1)]
TP['xgb_preds'].plot.hist(ax=ax, bins=300,facecolor='blue',alpha = 0.3, bottom=0.001)
ax.set_yscale('log')
plt.legend(('XGB predicted probability distribution','True positives = (MC =1) True lamdas in the distribution'), fontsize = 15, loc='upper right')
plt.title("XGB returned probablility for the 10k events data set", fontsize = 15)
plt.ylabel("Log scale number of counts", fontsize = 15)
plt.xlabel("Probability", fontsize = 15)
fig.tight_layout()
fig.savefig("hists.png")

In [ ]:
roc_auc=roc_auc_score(y_whole, df_clean['xgb_preds'])
fpr, tpr, thresholds = roc_curve(y_whole, df_clean['xgb_preds'],drop_intermediate=False ,pos_label=1)
S0 = sqrt(2 * ((tpr + fpr) * log(1 + tpr/fpr) - tpr))
S0 = S0[~np.isnan(S0)]
xi = argmax(S0)
S0_best_threshold = (thresholds[xi])
print('Best Threshold=%f, S0=%.3f' % (thresholds[xi], S0[xi]))

In [ ]:
labels = dtest.get_label()
print('error=%f' %
      (sum(1 for i in range(len(preds)) if int(preds[i] > 0.5) != labels[i]) /
       float(len(preds))))

In [ ]:
import treelite
model = treelite.Model.from_xgboost(bst)
toolchain = 'clang'
model.export_lib(toolchain=toolchain, libpath='./mymodel.so',
                 params={'parallel_comp': 8}, verbose=True)

# Operating system of the target machine
platform = 'unix'

# Save the source package as a zip archive named mymodel.zip
# Later, we'll use this package to produce the library mymodel.so.
model.export_srcpkg(platform=platform, toolchain=toolchain,
                    pkgpath='./mymodel.zip', libname='mymodel.so',
                    verbose=True)

In [ ]:
!unzip -l mymodel.zip

In [ ]:
import treelite_runtime     # runtime module
predictor = treelite_runtime.Predictor('./mymodel.so', verbose=True)

In [ ]:
x_whole_nump = pd.DataFrame.to_numpy(x_whole)

In [ ]:
batch = treelite_runtime.Batch.from_npy2d(x_whole_nump)
df_clean['tree_lite'] = predictor.predict(batch)

In [ ]:
plt.hist(out_pred,bins=300, range=(0.5,1) )
plt.show()

## Already working model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
xg_reg = xgb.XGBRegressor(learning_rate = 0.09273,
                max_depth = 9, alpha = 3.085, gamma =0.1481 , n_estimators = 115, nthread=8)

In [ ]:
train_pred = xg_reg.fit(x_train,y_train)

In [ ]:
preds = xg_reg.predict(x_test)

In [ ]:
df_clean['xgb_preds'] =xg_reg.predict(x_whole)

In [ ]:
df_clean['xgb_preds'].describe()

In [ ]:
AUC_score = roc_auc_score(y_train, xg_reg.predict(x_train))
print("AUC: %f" % (AUC_score))

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
roc_auc_score(y_test, preds)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_whole, df_clean['xgb_preds']))
print("RMSE: %f" % (rmse))

In [ ]:
roc_auc_score(y_whole, df_clean['xgb_preds'])

In [ ]:
ax = xgb.plot_importance(bst)
plt.rcParams['figure.figsize'] = [8, 4]
plt.show()
ax.figure.tight_layout() 
ax.figure.savefig("hits.png")

In [ ]:
def cut_visualization(cut, range1=(1.09, 1.19), bins1= 300 ):
    mask1 = df_clean['xgb_preds']>cut
    df3=df_clean[mask1]
    
    fig, ax2 = plt.subplots(figsize=(15, 10), dpi = 200)
    color = 'tab:blue'
    ax2.hist(df_clean['mass'],bins = bins1, range=range1, facecolor='blue',alpha = 0.35, label='before selection')
    ax2.set_ylabel('Counts', fontsize = 15, color=color)
    ax2.set_xlabel('Mass in GeV', fontsize = 15)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.legend( fontsize = 15, loc='upper left')
    
    color = 'tab:red'
    ax1 = ax2.twinx()
    ax1.hist(df3['mass'], bins = bins1, range=range1, facecolor='red',alpha = 0.35, label='Machine learning (XGB)')
    ax1.set_xlabel('Mass in GeV', fontsize = 15)
    ax1.set_ylabel('Counts ', fontsize = 15, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.legend( fontsize = 15,loc='upper right' )
    
    
    
    plt.title("The sample's Invariant Mass with XGB (with a cut > "+str(cut)+')', fontsize = 15)
    fig.tight_layout()
    #fig.savefig("whole_sample_invmass_with_ML.png")

In [ ]:
cut_visualization(S0_best_threshold)

In [ ]:
mask1 = df_clean['xgb_preds']>0.969618
df3=df_clean[mask1]
df3_clean_signal_mask = df3['issignal']>0
df3_clean_signal = df3[df_clean_signal_mask]
df3_clean_bac_mask = df3['issignal']==0
df3_clean_bac = df3[df_clean_bac_mask]
#vars_to_draw.remove('xgb_preds')



In [ ]:
mask10 = (df_clean['xgb_preds1']==1)&(df_clean['issignal']==1)
df4=df_clean[mask1]

In [ ]:
leg_labels = ['original background', 'background inside XGB predicted signal']
vars_to_draw = list(df3.columns)
ax = plot_utils.plot_distr([df_clean_bac, df3_clean_bac], vars_to_draw, bins=100, labels=leg_labels, log=True, density=True, figsize=(30, 20), alpha=0.3, grid=False)
plt.tight_layout()
plt.savefig("hits.png")

In [ ]:
cut3 = S0_best_threshold
mask1 = df_clean['xgb_preds']>cut3
df3=df_clean[mask1]
fig, axs = plt.subplots(figsize=(15, 10))

range1= (1.09, 1.7)

#xgb

df3_new = df3[df3['issignal']==1]
df3_new1 = df3[df3['issignal']==0]
df3['mass'].plot.hist(bins = 300, range=range1, facecolor='red',alpha = 0.3,grid=True,sharey=True)
#df3_new['mass'].plot.hist(bins = 300, range=range1,facecolor='blue',alpha = 0.3,grid=True,sharey=True)
df3_new1['mass'].plot.hist(bins = 300, range=range1,facecolor='green',alpha = 0.3,grid=True,sharey=True)
plt.legend(('XGB selected lambdas','\n False positives = \n (MC =0)\n background in \n the distribution' ), fontsize = 18, loc='upper right')
#plt.rcParams["legend.loc"] = 'upper right'
plt.title("KFPF variables + cos$\Theta_{between \ \overrightarrow{P_\Lambda} \  & \ \overrightarrow{P_{\Pi^-}}}$ + $P_T$  with a cut of %.4f "%cut3 +"on the XGB probability distribution", fontsize = 18)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 18)
plt.ylabel("Counts", fontsize = 18)
axs.tick_params(labelsize=18)
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

## Confusion Matrix

By definition a confusion matrix $C$ is such that $C_{i, j}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$.

Thus in binary classification, the count of true positives is $C_{0,0}$, false positives is $C_{1,0}$, true negatives is $C_{1,1}$ and false negatives is $C_{0,1}$.

The following function prints and plots the confusion matrix. Normalization can be applied by setting `normalize=True`.

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',fontsize = 15)
    plt.xlabel('Predicted label',fontsize = 15)

In [ ]:
cut1 = S0_best_threshold
df_clean['xgb_preds1'] = ((df_clean['xgb_preds']>cut1)*1)
cnf_matrix = confusion_matrix(y_whole, df_clean['xgb_preds1'], labels=[1,0])
np.set_printoptions(precision=2)
fig, axs = plt.subplots(figsize=(10, 8))
axs.yaxis.set_label_coords(-0.04,.5)
axs.xaxis.set_label_coords(0.5,-.005)
plot_confusion_matrix(cnf_matrix, classes=['signal','background'], title='Confusion Matrix for XGB for cut > '+str(cut1))
plt.savefig('confusion_matrix_extreme_gradient_boosting_whole_data.png')

In [ ]:
def pT_vs_rapidity(var_yaxis, var_xaxis, range_var_yaxis, range_var_xaxis):
    import matplotlib as mpl
    fig, axs = plt.subplots(figsize=(15, 10))
    plt.hist2d(var_yaxis,var_xaxis,range=[range_var_yaxis,range_var_xaxis], bins=100, norm=mpl.colors.LogNorm(), cmap=plt.cm.Greys)
    plt.xlabel('pT', fontsize=15)
    plt.ylabel('rapidity', fontsize=15)
    plt.show()
    fig.tight_layout()
    fig.savefig("/home/shahid/cbmsoft/Cut_optimization/uncut_data/pT_vs_rapidity.png")

In [ ]:
new_df=df_clean[df_clean['xgb_preds1']==1] 
range1=[-0.1, 3.5]
range2=[-0.5, 3]

pT_vs_rapidity(new_df['rapidity'], new_df['pT'], range1, range2)
#fig.savefig("/home/shahid/cbmsoft/Cut_optimization/uncut_data/pT_vs_rapidity_xgb.png")

In [ ]:
df3_new = new_df[new_df['issignal']==1]
range1=[-0.1, 3.5]
range2=[-0.5, 3]

pT_vs_rapidity(df3_new['rapidity'], df3_new['pT'], range1, range2)

#fig.savefig("/home/shahid/cbmsoft/Cut_optimization/uncut_data/pT_vs_rapidity_xgb_true.png")

In [ ]:
new_df1=df_clean[df_clean['issignal']==1]
range1=[-0.1, 3.5]
range2=[-0.5, 3]

pT_vs_rapidity(new_df1['rapidity'], new_df1['pT'], range1, range2)
#fig.savefig("/home/shahid/cbmsoft/Cut_optimization/uncut_data/pT_vs_rapidity.png")

In [ ]:
np.exp(-1.03)/(1+np.exp(-1.03))

In [ ]:
import seaborn as sb

In [ ]:
signal_selected['cosineneg'].median()

In [ ]:
cuts1 =['pz','cosineneg']
bac_new = background_selected[background_selected['mass']>1.1227]

In [ ]:
bac_new['cosineneg'].median()

In [ ]:
sb_plot = sb.pairplot(bac_new,vars = cuts1, height=5)
sb_plot.savefig("output.png")

In [ ]:
cut1

In [ ]:
df_clean['kfpf_preds']=0
df_clean['kfpf_preds'] = (df_clean['chi2primpos'] > 18.4)*1
df_clean['kfpf_preds'] =(df_clean['chi2primneg'] > 18.4)*1
df_clean['kfpf_preds'] = (df_clean['ldl'] > 5)*1
df_clean['kfpf_preds'] = (df_clean['distance'] < 1)*1
df_clean['kfpf_preds'] = (df_clean['chi2geo'] < 3)*1
df_clean['kfpf_preds'] = (df_clean['cosinepos'] > 0)*1
df_clean['kfpf_preds'] = (df_clean['cosineneg'] > 0)*1
df_clean['kfpf_preds']

In [ ]:

cnf_matrix = confusion_matrix(new_check_set['issignal'], new_check_set['new_signal'], labels=[1,0])
np.set_printoptions(precision=2)
fig, axs = plt.subplots(figsize=(10, 8))
axs.yaxis.set_label_coords(-0.04,.5)
axs.xaxis.set_label_coords(0.5,-.005)
plot_confusion_matrix(cnf_matrix, classes=['signal','background'], title='Confusion Matrix for XGB for cut > '+str(cut1))
plt.savefig('confusion_matrix_extreme_gradient_boosting_whole_data.png')

In [ ]:
10725/np.sqrt(30746)


#[[   10755     2154]
 #[   28054 20686209]]
10755/np.sqrt(20686209+10755)

#[[   10571     2338]
# [   26070 20688193]]
10571/np.sqrt(20688193+10571)

In [ ]:
roc_auc=roc_auc_score(y_whole, df_clean['xgb_preds'])
fpr, tpr, thresholds = roc_curve(y_whole, df_clean['xgb_preds'],drop_intermediate=False ,pos_label=1)
print(roc_auc)

In [ ]:
S0 = sqrt(2 * ((tpr + fpr) * log((1 + tpr/fpr)) - tpr))
S0 = S0[~np.isnan(S0)]
xi = argmax(S0)
S0_best_threshold = (thresholds[xi])
print('Best Threshold=%f, S0=%.3f' % (thresholds[xi], S0[xi]))

In [ ]:

# calculate the geometric-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = argmax(gmeans)
a = str(thresholds[ix])
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

In [ ]:
def significance(y_true, y_predict):
    roc_auc=roc_auc_score(y_true, y_predict)
    fpr, tpr, thresholds = roc_curve(y_true, y_predict,drop_intermediate=False ,pos_label=1)
    S0 = sqrt(2 * ((tpr + fpr) * log((1 + tpr/fpr)) - tpr))
    S0 = S0[~np.isnan(S0)]
    xi = argmax(S0)
    S0_best_threshold = (thresholds[xi])
    print('Best Threshold=%f, S0=%.3f' % (thresholds[xi], S0[xi]))
    print('AUC score : ', roc_auc)
    return fpr, tpr, S0_best_threshold, S0[xi], xi

    

In [ ]:
significance(y_whole, preds)

In [ ]:
roc_auc=roc_auc_score(y_whole, preds)
fpr, tpr, thresholds = roc_curve(y_whole, preds,drop_intermediate=False ,pos_label=1)
S0 = sqrt(2 * ((tpr + fpr) * log(1 + tpr/fpr) - tpr))
S0 = S0[~np.isnan(S0)]
xi = argmax(S0)
S0_best_threshold = (thresholds[xi])
print('Best Threshold=%f, S0=%.3f' % (thresholds[xi], S0[xi]))

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10), dpi = 100)
plt.plot(fpr, tpr, color='darkorange',
          label='ROC curve (area = %0.4f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.scatter(fpr[xi], tpr[xi], marker='o', color='black', label= 'Best Threshold = '+"%.4f" % S0_best_threshold +'\n S0 = '+ "%.2f" % S0[xi])
plt.xlabel('False Positive Rate', fontsize = 15)
plt.ylabel('True Positive Rate', fontsize = 15)
plt.legend(loc="lower right", fontsize = 15)
plt.title('Receiver operating characteristic', fontsize = 15)
plt.xlim([-0.01, 1.0])
plt.ylim([0, 1.02])
#axs.axis([-0.01, 1, 0.9, 1])
fig.tight_layout()
plt.savefig('ROC_and_AUC.png')

<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>Tree visualization
<br></p><br>

In [ ]:
#fig, ax1 = plt.subplots(figsize=(15, 10), dpi = 200)
xgb.plot_tree(xg_reg,num_trees=10)
plt.rcParams['figure.figsize'] = [20, 40]
plt.rcParams['figure.dpi']=200
plt.show()

In [ ]:
xgb.to_graphviz(xg_reg, fmap='', num_trees=0, rankdir=None, yes_color=None, no_color=None, condition_node_params=None, leaf_node_params=None)

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import matplotlib.pyplot as plt

# Define objective function
def f(x):
    return {'loss': x ** 2 - x, 'status': STATUS_OK}

# Run hyperopt optimization
trials = Trials()
result = fmin(
    fn=f,                           # objective function
    space=hp.uniform('x', -1, 1),   # parameter space
    algo=tpe.suggest,               # surrogate algorithm
    max_evals=500,                  # no. of evaluations
    trials=trials                   # trials object that keeps track of the sample results (optional)
)



# Print the optimized parameters
print(result)   # {'x': 0.5000833960783931}

# Extract and plot the trials 
x = trials.vals['x']
y = [x['loss'] for x in trials.results]
plt.scatter(x, y)

In [ ]:
from sklearn.metrics import mean_squared_error


# XGB parameters
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials


class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [ ]:
obj = HPOpt(x_train, x_test, y_train, y_test)

xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)


In [ ]:
print(result)

In [ ]:
model_clf = xgb.XGBClassifier()
model_hdl = ModelHandler(model_clf, cuts)
train_test_data = train_test_generator([y, x], [1,0], test_size=0.5, random_state=42)

In [ ]:
hyper_pars_ranges = {'n_estimators': (200, 1000), 'max_depth': (2, 4), 'learning_rate': (0.01, 0.1)}
model_hdl.optimize_params_bayes(df_, hyper_pars_ranges, 'roc_auc', nfold=5, init_points=5, n_iter=5, njobs=-1)

# Animation

In [ ]:
"""
Matplotlib Animation Example

author: Jake Vanderplas
email: vanderplas@astro.washington.edu
website: http://jakevdp.github.com
license: BSD
Please feel free to use and modify this, but keep the above information. Thanks!
"""

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-2, 2))
line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# animation function.  This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)

# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()


def f(x, y):
    return np.sin(x) + np.cos(y)

x = np.linspace(0, 2 * np.pi, 120)
y = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

# ims is a list of lists, each row is a list of artists to draw in the
# current frame; here we are just animating one artist, the image, in
# each frame
ims = []
for i in range(60):
    x += np.pi / 15.
    y += np.pi / 20.
    im = ax.imshow(f(x, y), animated=True)
    if i == 0:
        ax.imshow(f(x, y))  # show an initial one first
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)

# To save the animation, use e.g.
#
# ani.save("movie.mp4")
#
# or
#
writer = animation.FFMpegWriter(fps=15, metadata=dict(artist='Me'), bitrate=1800)
ani.save("movie.mp4", writer=writer)

plt.show()

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import matplotlib.animation as animation

# Fixing random state for reproducibility
np.random.seed(19680801)

# histogram our data with numpy
data = np.random.randn(1000)
n, bins = np.histogram(data, 100)

# get the corners of the rectangles for the histogram
left = bins[:-1]
right = bins[1:]
bottom = np.zeros(len(left))
top = bottom + n
nrects = len(left)

In [ ]:
nverts = nrects * (1 + 3 + 1)
verts = np.zeros((nverts, 2))
codes = np.full(nverts, path.Path.LINETO)
codes[0::5] = path.Path.MOVETO
codes[4::5] = path.Path.CLOSEPOLY
verts[0::5, 0] = left
verts[0::5, 1] = bottom
verts[1::5, 0] = left
verts[1::5, 1] = top
verts[2::5, 0] = right
verts[2::5, 1] = top
verts[3::5, 0] = right
verts[3::5, 1] = bottom

In [ ]:
patch = None


def animate(i):
    # simulate new data coming in
    data = np.random.randn(1000)
    n, bins = np.histogram(data, 100)
    top = bottom + n
    verts[1::5, 1] = top
    verts[2::5, 1] = top
    return [patch, ]

In [ ]:
fig, ax = plt.subplots()
barpath = path.Path(verts, codes)
patch = patches.PathPatch(
    barpath, facecolor='green', edgecolor='yellow', alpha=0.5)
ax.add_patch(patch)

ax.set_xlim(left[0], right[-1])
ax.set_ylim(bottom.min(), top.max())

ani = animation.FuncAnimation(fig, animate, 50, repeat=False, blit=True)
plt.show()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from celluloid import Camera

fig, axes = plt.subplots(2)
camera = Camera(fig)
t = np.linspace(0, 2 * np.pi, 128, endpoint=False)
for i in t:
    axes[0].plot(t, np.sin(t + i), color='blue')
    axes[1].plot(t, np.sin(t - i), color='blue')
    camera.snap()
    
animation = camera.animate()  
animation.save('celluloid_subplots.gif', writer = 'imagemagick')


In [ ]:
n_bns = 1000

fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15,10))
ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7 = axes.flatten()

ax0.hist(true_signal.iloc[:,0], bins=n_bns, alpha=0.2, facecolor='red', stacked=True, log= True,label='Signal')
ax0.hist(true_bac.iloc[:,0], bins=n_bns, alpha=0.2, facecolor='blue', stacked=True, log= True,label='Background')
ax0.legend(prop={'size': 10})
ax0.set_title('chi2 geometric')


ax1.hist(true_signal.iloc[:,1], bins=n_bns, alpha=0.2, facecolor='red',range=(100,1000000),stacked=True, log= True,label='Signal')
ax1.hist(true_bac.iloc[:,1], bins=n_bns, alpha=0.2, facecolor='blue',range=(100,1000000),stacked=True, log= True,label='Background')
ax1.set_title('chi2 primary pion')
ax1.legend(prop={'size': 10})

ax2.hist(true_signal.iloc[:,2], bins=n_bns, alpha=0.2, facecolor='red',range=(100,100000),stacked=True, log= True,label='Signal')
ax2.hist(true_bac.iloc[:,2], bins=n_bns, alpha=0.2, facecolor='blue',range=(100,100000),stacked=True, log= True,label='Background')
ax2.set_title('chi2 primary proton')
ax2.legend(prop={'size': 10})

ax3.hist(true_signal.iloc[:,3], bins=n_bns, alpha=0.2, facecolor='red',range=(-1,30),stacked=True, log= True,label='Signal')
ax3.hist(true_bac.iloc[:,3], bins=n_bns, alpha=0.2, facecolor='blue',range=(-1,30),stacked=True, log= True,label='Background')
ax3.set_title('chi2 topological')
ax3.legend(prop={'size': 10})


ax4.hist(true_signal.iloc[:,6], bins=n_bns, alpha=0.2, facecolor='red',range=(0,5000),stacked=True, log= True,label='Signal')
ax4.hist(true_bac.iloc[:,6], bins=n_bns, alpha=0.2, facecolor='blue',range=(0,5000),stacked=True, log= True,label='Background')
ax4.set_title('l/dl')
ax4.legend(prop={'size': 10})

ax5.hist(true_signal.iloc[:,14], bins=n_bns, alpha=0.2, facecolor='red', stacked=True, log= True,label='Signal')
ax5.hist(true_bac.iloc[:,14], bins=n_bns, alpha=0.2, facecolor='blue', stacked=True, log= True,label='Background')
ax5.set_title('Distance between secondary tracks (corr= 0.178)')
ax5.legend(prop={'size': 10})

ax6.hist(true_signal.iloc[:,17], bins=n_bns, alpha=0.2, facecolor='red', stacked=True, log= True,label='Signal')
ax6.hist(true_bac.iloc[:,17], bins=n_bns, alpha=0.2, facecolor='blue', stacked=True, log= True,label='Background')
ax6.set_title('cosine of angle between proton and lambda (corr= -0.193)')
ax6.legend(prop={'size': 10})

ax7.hist(true_signal.iloc[:,16], bins=n_bns, alpha=0.2, facecolor='red',range=(0.65,1),stacked=True, log= True,label='Signal')
ax7.hist(true_bac.iloc[:,16], bins=n_bns, alpha=0.2, facecolor='blue',range=(0.65,1),stacked=True, log= True,label='Background')
ax7.set_title('cosine of angle between pion and lambda (corr=-0.25)')
ax7.legend(prop={'size': 10})

fig.tight_layout()
plt.show()
fig.savefig("KFPF_chi2.png")

In [ ]:
#terminal      conda activate myrootenv
#terminal      root --notebook
import ROOT

In [ ]:
h = ROOT.TH1F("myHist", "myTitle", 64, -4, 4)
h.FillRandom("gaus")

In [ ]:
f = ROOT.TF1("f1", "sin(x)/x", 0., 10.)
f.Draw()

# Cut visualization

In [ ]:
# The following function will display the inavriant mass histogram of the original 10k event set along with the mass histoigram after we apply a cut
# on the probability prediction of xgb
def cut_visualization(cut, range1=(1.09, 1.19), bins1= 300 ):
    mask1 = df_clean['xgb_preds']>cut
    df3=df_clean[mask1]
    
    fig, ax2 = plt.subplots(figsize=(15, 10), dpi = 200)
    color = 'tab:blue'
    ax2.hist(new_check_set['mass'],bins = bins1, range=range1, facecolor='blue',alpha = 0.35, label='KFPF')
    ax2.set_ylabel('Counts', fontsize = 15, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.legend( fontsize = 15, loc='upper left')
    
    color = 'tab:red'
    ax1 = ax2.twinx()
    ax1.hist(df3['mass'], bins = bins1, range=range1, facecolor='red',alpha = 0.35, label='XGB')
    ax1.set_xlabel('Mass in GeV', fontsize = 15)
    ax1.set_ylabel('Counts ', fontsize = 15, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.legend( fontsize = 15,loc='upper right' )

    plt.title("The original sample's Invariant Mass along with mass after selection of XGB (with a cut > "+str(cut)+')', fontsize = 15)
    fig.tight_layout()
    fig.savefig("hists.png")

In [ ]:
cut_visualization(0)

In [ ]:
new_check_set= df_clean.copy()
new_check_set['new_signal']=0
mask1 = (new_check_set['chi2primpos'] > 18.4) & (new_check_set['chi2primneg'] > 18.4)

mask2 = (new_check_set['ldl'] > 5) & (new_check_set['distance'] < 1)

mask3 = (new_check_set['chi2geo'] < 3) & (new_check_set['cosinepos'] > 0) & (new_check_set['cosineneg'] > 0)

new_check_set = new_check_set[(mask1) & (mask2) & (mask3)] 

#After all these cuts, what is left is considered as signal, so we replace all the values in the 'new_signal'
# column by 1
new_check_set['new_signal'] = 1
new_check_set

In [ ]:
cut3 = S0_best_threshold
mask1 = df_clean['xgb_preds']>cut3
df3=df_clean[mask1]
fig, axs = plt.subplots(figsize=(10, 6))

range1= (1.09, 1.17)

#xgb


(df3['mass']).plot.hist(bins = 300, range=range1, facecolor='red',alpha = 0.3,grid=True,sharey=True)
(new_check_set['mass']).plot.hist(bins = 300, range=range1,facecolor='blue',alpha = 0.3,grid=True,sharey=True)
plt.xlabel("Mass in GeV", fontsize = 15)
plt.ylabel("counts", fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(('XGBoost','KFPF'), fontsize = 15, loc='upper left')
#plt.rcParams["legend.loc"] = 'upper right'
plt.title("The lambda's Invariant Mass histogram with KFPF and XGB selection criteria", fontsize = 15)
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

In [ ]:
from matplotlib import gridspec

range1= (1.09, 1.17)


fig, axs = plt.subplots(2, 1,figsize=(15,10), sharex=True,  gridspec_kw={'width_ratios': [10],
                           'height_ratios': [8,4]})

ns, bins, patches=axs[0].hist((df3_base['mass']),bins = 300, range=range1, facecolor='red',alpha = 0.3)
ns1, bins1, patches1=axs[0].hist((new_check_set['mass']),bins = 300, range=range1,facecolor='blue',alpha = 0.3)
#plt.xlabel("Mass in GeV", fontsize = 15)
axs[0].set_ylabel("counts", fontsize = 15)
#axs[0].grid()
axs[0].legend(('XGBoost Selected $\Lambda$s','KFPF selected $\Lambda$s'), fontsize = 15, loc='upper right')

#plt.rcParams["legend.loc"] = 'upper right'
axs[0].set_title("The lambda's Invariant Mass histogram with KFPF and XGB selection criteria on KFPF variables", fontsize = 15)
axs[0].grid()
axs[0].tick_params(axis='both', which='major', labelsize=15)
#fig.savefig("whole_sample_invmass_with_ML.png")


hist1, bin_edges1 = np.histogram(df3_base['mass'],range=(1.09, 1.17), bins=300)
hist2, bin_edges2 = np.histogram(new_check_set['mass'],range=(1.09, 1.17), bins=300)

#makes sense to have only positive values 
diff = (hist1 - hist2)
axs[1].bar(bins[:-1],     # this is what makes it comparable
        ns / ns1, # maybe check for div-by-zero!
        width=0.001)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 15)
axs[1].set_ylabel("XGB / KFPF", fontsize = 15)
axs[1].grid()
axs[1].tick_params(axis='both', which='major', labelsize=15)

plt.show()
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

In [ ]:
cut3 = S0_best_threshold
mask1 = df_clean['xgb_preds']>cut3
df3_base_cospos=df_clean[mask1]

In [ ]:
cut3 = S0_best_threshold
mask1 = df_clean['xgb_preds']>cut3
df3_base=df_clean[mask1]

In [ ]:
from matplotlib import gridspec

range1= (1.09, 1.17)
bin1 = 200


fig, axs = plt.subplots(2, 1,figsize=(15,10), sharex=True,  gridspec_kw={'width_ratios': [10],
                           'height_ratios': [8,4]})

ns, bins, patches=axs[0].hist((df3_base_cospos['mass']),bins = bin1, range=range1, facecolor='red',alpha = 0.3)
ns1, bins1, patches1=axs[0].hist((df3_base['mass']),bins = bin1, range=range1,facecolor='blue',alpha = 0.3)
#plt.xlabel("Mass in GeV", fontsize = 15)
axs[0].set_ylabel("counts", fontsize = 15)
#axs[0].grid()
axs[0].legend(('XGB base+ cos$\Theta_{between \ \overrightarrow{P_\Lambda} \  & \ \overrightarrow{P_{proton}}}$ Selected $\Lambda$s','XGB base selected $\Lambda$s'), fontsize = 15, loc='upper right')

#plt.rcParams["legend.loc"] = 'upper right'
axs[0].set_title("The lambda's Invariant Mass histogram with XGB base and XGB base+ cos$\Theta_{between \ \overrightarrow{P_\Lambda} \  & \ \overrightarrow{P_{proton}}}$ selection criteria", fontsize = 15)
axs[0].grid()
axs[0].tick_params(axis='both', which='major', labelsize=15)
#fig.savefig("whole_sample_invmass_with_ML.png")


hist1, bin_edges1 = np.histogram(df3_base_cospos['mass'],range=range1, bins=bin1)
hist2, bin_edges2 = np.histogram(df3_base['mass'],range=range1, bins=bin1)

#makes sense to have only positive values 
diff = (hist1 - hist2)
axs[1].bar(bins[:-1],     # this is what makes it comparable
        ns / ns1, # maybe check for div-by-zero!
        width=0.001)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 15)
axs[1].set_ylabel("XGB base+cospos / XGB base", fontsize = 15)
axs[1].grid()
axs[1].tick_params(axis='both', which='major', labelsize=15)

plt.show()
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")